In [2]:
!git clone https://github.com/AminMohamed-3/Emotion-Classification.git
!pip install transformers dataset accelerate -q
import sys
sys.path.append("/kaggle/working/Emotion-Classification")

fatal: destination path 'Emotion-Classification' already exists and is not an empty directory.


In [3]:
import torch
from Training.dataset import prepare_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    DataCollatorForTokenClassification
)
import numpy as np
from config import NUM_LABELS
import wandb
from Training.utils import compute_metrics
from Training.utils import MultiLabelTrainer

# Define the model & Prepare Dataset

In [2]:
model_checkpoint = "FacebookAI/roberta-base" # Using a larger model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
dataset, id2label, label2id = prepare_dataset(tokenizer)

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=NUM_LABELS, id2label=id2label, label2id=label2id
)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

100%|██████████| 211225/211225 [00:01<00:00, 132837.53it/s]


Map:   0%|          | 0/168980 [00:00<?, ? examples/s]

Map:   0%|          | 0/21122 [00:00<?, ? examples/s]

Map:   0%|          | 0/21123 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Trainer

In [7]:
data_collator = DataCollatorForTokenClassification(tokenizer, padding=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,  # Training for longer
    per_device_train_batch_size=16, # smaller batch size
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5, # Using a smaller LR
    save_total_limit=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=20,
    # weight decay
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
)


In [25]:
trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
wandb.login(key="62f8ddd1a44f05efc5c27f0ee5f22cf5bd70abc5")
trainer.train()